# Multimodal Financial Research Agent with Hybrid Search
Build an intelligent agent that can search financial documents (RAG) and live market data to provide comprehensive insights.

## Overview

This notebook covers:
- **Hybrid Search (RAG)**: Semantic + keyword search with automatic filter extraction from historical SEC filings
- **Live Finance Research**: Real-time stock data and market information via Yahoo Finance MCP
- **Agent with Dual Tools**: Connect LLM agent with both vector database and live APIs
- **Financial Analysis**: Query historical SEC filings (10-K, 10-Q) and current market data
- **Metadata Filtering**: Automatic extraction of company, year, quarter from queries

## Setup

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.agents import create_agent

from langchain.messages import HumanMessage, ToolMessage, AIMessage, SystemMessage
from langchain_core.tools import tool

from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3


In [ ]:
from scripts.rag_tools import hybrid_search
from scripts.prompts import MULTIMODEL_AGENT_PROMPT

In [6]:
# Initialize Model
model = ChatGoogleGenerativeAI(model='gemini-3-flash-preview')

In [7]:
import subprocess
import sys

@tool
def live_finance_researcher(query: str):
    """
    Research live stock data using Yahoo Finance MCP.
    
    Use this tool to get:
    - Current stock prices and real-time market data
    - Latest financial news
    - Stock recommendations and analyst ratings
    - Option chains and expiration dates
    - Recent stock actions (splits, dividends)
    
    Args:
        query: The financial research question about current market data
        
    Returns:
        Research results from Yahoo Finance
    """

    code = f"""
import asyncio
from scripts.yahoo_mcp import finance_research
asyncio.run(finance_research("{query}"))
"""
    result = subprocess.run([sys.executable, '-c', code], capture_output=True, text=True)

    return result.stdout

## Create Financial Research Agent with Memory

In [19]:
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3
from langchain.agents import create_agent

def get_agent():
    conn = sqlite3.connect("data/financial_research_agent.db", check_same_thread=False)
    checkpointer = SqliteSaver(conn=conn)

    agent = create_agent(
        model=model,
        tools=[hybrid_search, live_finance_researcher],
        system_prompt=MULTIMODAL_AGENT_PROMPT,
        checkpointer=checkpointer
    )

    return agent

In [20]:
config = {"configurable": {"thread_id": "session1"}}
query = "What is the apple's cash flow in 2023?"

agent = get_agent()
agent.invoke({"messages": [HumanMessage(query)]}, config=config)

{'messages': [HumanMessage(content="What is the apple's cash flow in 2023?", additional_kwargs={}, response_metadata={}, id='99c40184-e79e-445c-beeb-353c54da29a7'),
  AIMessage(content=[], additional_kwargs={'function_call': {'name': 'hybrid_search', 'arguments': '{"query": "Apple cash flow 2023 10-K"}'}, '__gemini_function_call_thought_signatures__': {'5170527b-2e6d-4edc-84f3-b8690013c23f': 'EpgECpUEAb4+9vuWt/Tgr7kcfgTEQlk+5tHLGMtJrxPW7QO2khxGbg5L3vyWOdqWwuakfs4PKm3lN/aewStsFkBcabNDVgthPSrGmc/TDEPQQNabgLdHGOXZFyfDzRst9o6brENuHhX8JJNqa1YgNTIiALlTRZ+cNuQEtlmvi3PJYqluh8KCdRC3gnP6CqsiXW00gnVTgT/WPBqyPcBRZmpitxGX5StMSE0BqBUT2QesAQ/CAc5VUiov19hoxLoKndxp9w7HAJPCuvLHqLPfNsVpw7mG9xXmLmeftp4FNWGjAYFnhRI2O1CCVjJBjvz9SoyoiL8/umj6ynC/ATy9ZZt08rs0foKdJAK/wvpYYQb2kKqcJHwjHFq8ML4azsiuwsaAOonyJHw6XESzVffEka2nIqtDYEe0m0wG7E5WN1KfoVRvRSfy7kcL3eDZklE7vg/YtVzSm/ijDBEjwz4kTdqU0FEeTek/ouNXAM0J/nSDC2zC1FrQtgky3SemjOMRrE0T9nIy40uUoFa++mnBgVlzZgzmnSUG0iP54jBdBK/a/yZeCxnOQRiUMJGxjcRCeI0cadKN+WPvpvSe+/gxEmEu/zYg

In [21]:
response = agent.invoke({"messages": [HumanMessage("Which questions I've just ask?")]}, config=config)

In [24]:
response["messages"][-1].text

"You have asked the following question in this conversation:\n\n1. **What is the apple's cash flow in 2023?**"

## Streaming Helper Function

In [25]:
def stream_agent_response(agent, query, thread_id="default"):

    config = {'configurable': {'thread_id': thread_id}}
    
    for chunk in agent.stream(
        {'messages': [HumanMessage(query)]},
        stream_mode='messages',
        config=config
    ):
        # Extract message from chunk
        message = chunk[0] if isinstance(chunk, tuple) else chunk
        
        # Handle AI messages with tool calls
        if isinstance(message, AIMessage) and message.tool_calls:
            for tool_call in message.tool_calls:
                print(f"\n  Tool Called: {tool_call['name']}")
                print(f"   Args: {tool_call['args']}")
                print()
        
        # Handle tool responses
        elif isinstance(message, ToolMessage):
            print(f"\n  Tool Result (length: {len(message.text)} chars)")
            print()
        
        # Handle AI text responses
        elif isinstance(message, AIMessage) and message.text:
            # Stream the text content
            print(message.text, end='', flush=True)

## Examples

In [26]:
query = 'What is the apples cash flow in 2023?'
agent = get_agent()

stream_agent_response(agent, query, 'session2')


  Tool Called: hybrid_search
   Args: {'query': 'Apple 2023 cash flow'}

{"company_name": "apple", "doc_type": null, "fiscal_year": "2023", "fiscal_quarter": null}
  Tool Result (length: 26471 chars)

For the fiscal year ended September 30, 2023, Apple Inc. reported the following cash flow results:

### **Apple Inc. Cash Flow Summary (Fiscal Year 2023)**
(All figures in millions)

| Category | Amount |
| :--- | :--- |
| **Cash generated by operating activities** | **$110,543** |
| **Cash generated by investing activities** | **$3,705** |
| **Cash used in financing activities** | **$(108,488)** |
| **Net increase in cash, cash equivalents, and restricted cash** | **$5,760** |

---

### **Key Highlights from 2023 Cash Flows:**
*   **Operating Activities:** Apple generated over $110.5 billion in cash from operations, driven primarily by net income of $96.99 billion, adjusted for $11.5 billion in depreciation and amortization and $10.8 billion in share-based compensation.
*   **Investing 

In [27]:
agent = get_agent()
stream_agent_response(agent, "What was Amazon's revenue in Q1 2024?", thread_id="session3")


  Tool Called: hybrid_search
   Args: {'query': 'Amazon revenue Q1 2024'}

{"company_name": "amazon", "doc_type": "10-q", "fiscal_year": "2024", "fiscal_quarter": "q1"}
  Tool Result (length: 23975 chars)

Amazon's consolidated revenue (net sales) for the first quarter of 2024, which ended March 31, 2024, was **$143.31 billion**. This represents a 13% increase compared to the $127.36 billion reported in Q1 2023.

### Revenue Breakdown by Segment (Q1 2024)

| Segment | Revenue (in millions) | Year-over-Year Growth |
| :--- | :--- | :--- |
| **North America** | $86,341 | 12% |
| **International** | $31,935 | 10% |
| **AWS (Amazon Web Services)** | $25,037 | 17% |
| **Consolidated Total** | **$143,313** | **13%** |

The growth in sales across these segments was primarily driven by increased customer usage, unit sales, advertising sales, and a continued focus on selection and convenience for customers.

Source: amazon 10-q q1 2024.md, page 25

In [28]:
agent = get_agent()
stream_agent_response(agent, "What is the current stock price of Apple (AAPL) and show me latest news?",
                      thread_id="session4")


  Tool Called: live_finance_researcher
   Args: {'query': 'current stock price and latest news for Apple (AAPL)'}


  Tool Result (length: 2058 chars)

As of the latest market data, the current stock price for **Apple Inc. (AAPL)** is **$260.38**, representing a decrease of **0.52%** from its previous close.

### **Stock Price Details**
| Metric | Value |
| :--- | :--- |
| **Current Price** | $260.38 |
| **Previous Close** | $261.73 |
| **Today's Range** | $259.65 - $262.23 |
| **Change** | -1.35 (-0.52%) |

### **Latest News Highlights**
*   **Regulatory Scrutiny:** Apple is currently facing questioning from the Federal Trade Commission (FTC) regarding the political neutrality of Apple News.
*   **AI and Product Delays:** There are reports of delays for major AI-driven upgrades to Siri due to technical issues, which has contributed to investor nervousness.
*   **Tech Sector Selloff:** Apple shares have been impacted by a broader tech selloff driven by mounting fears over AI disruptio

In [29]:
agent = get_agent()
stream_agent_response(agent, "Compare Microsoft's Q2 2024 revenue from filings with its current stock performance",
                      thread_id="session5")


  Tool Called: hybrid_search
   Args: {'query': 'Microsoft Q2 2024 revenue (quarter ending December 31, 2023)'}


  Tool Called: live_finance_researcher
   Args: {'query': 'Microsoft (MSFT) current stock price and performance news'}

{"company_name": "microsoft", "doc_type": "10-q", "fiscal_year": "2024", "fiscal_quarter": "q2"}
  Tool Result (length: 0 chars)


  Tool Result (length: 1725 chars)


  Tool Called: hybrid_search
   Args: {'query': 'Microsoft fiscal second quarter 2024 revenue 10-Q December 31 2023'}

{"company_name": "microsoft", "doc_type": "10-q", "fiscal_year": "2024", "fiscal_quarter": "q2"}
  Tool Result (length: 0 chars)


  Tool Called: hybrid_search
   Args: {'query': 'Microsoft revenue for the three months ended December 31 2023'}

{"company_name": "microsoft", "doc_type": "10-q", "fiscal_year": "2023", "fiscal_quarter": "q4"}
  Tool Result (length: 0 chars)


  Tool Called: hybrid_search
   Args: {'query': 'Microsoft 10-K June 30 2024 revenue'}

{"company_name

In [30]:
agent = get_agent()
stream_agent_response(agent, "Compare Google's revenue between Q1 2024 and Q1 2023", 
                      thread_id="session6")


  Tool Called: hybrid_search
   Args: {'query': 'Alphabet Q1 2024 revenue and Q1 2023 revenue comparison'}

{"company_name": "google", "doc_type": "10-q", "fiscal_year": null, "fiscal_quarter": "q1"}
  Tool Result (length: 19200 chars)


  Tool Called: hybrid_search
   Args: {'query': 'Alphabet revenue Q1 2023 and Q1 2024 comparison 10-Q'}

{"company_name": "google", "doc_type": "10-q", "fiscal_year": null, "fiscal_quarter": "q1"}
  Tool Result (length: 25027 chars)


  Tool Called: hybrid_search
   Args: {'query': 'Alphabet total revenue Q1 2023 10-Q'}

{"company_name": "google", "doc_type": "10-q", "fiscal_year": "2023", "fiscal_quarter": "q1"}
  Tool Result (length: 0 chars)


  Tool Called: hybrid_search
   Args: {'query': 'Alphabet Q1 2023 revenues table'}

{"company_name": "google", "doc_type": "10-q", "fiscal_year": "2023", "fiscal_quarter": "q1"}
  Tool Result (length: 0 chars)


  Tool Called: live_finance_researcher
   Args: {'query': 'Alphabet (Google) revenue Q1 2023 and Q